 #  Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)


## Introduction/Business Problem

The question I want to answer in this project is the following: is there a correlation between the crime rate and the avenues in New York boroughs.  It is know that the crime, in general, is highly correlated to many factors such as poverty, unemployment, low educational attainment. However, there might be a correlation between crime rate and the type of avenues in certain areas. For example, areas with high crime rate might have more clubs, bars whereas area with low crime rate might have more museums and parks. I am not saying that this is the truth but the project will allow me to check this hypothesis and determine this correlation. 
Once we determine the avenues that are more prevalent in safe areas, we could probably generalize the correlation to areas where crime data is not available and predict the safety level of the area based on the type of avenues. I believe this analysis will be fun and interesting and I am excited to see the results.
The audience could be NY city authorities, social scientists, and criminologists. This information will help them better understand factors that govern the crime rate in the city and how we can control the crime rate by controlling the type of avenues.  


## Data 

I will use 2 source of data:
1.	Crime map based on NYPD complaint data current from the NYC open data (https://data.cityofnewyork.us/Public-Safety/Crime-Map-/5jvd-shfj). This dataset includes all valid felony, misdemeanor, and violation crimes reported to the New York City Police Department (NYPD) for all complete quarters from 1974 till 2020. It has 108058 crimes with information about type of the crime, borough, date and time, and location (lat, lon). That data can be easily exported as csv and used for analysis.
2.	I will use the Fousquare to determine the avenue in different NY borough and other information. Avenue extraction is similar to the practices we had. 


In [597]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 


Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4
  - defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4done

# All requested packages already installed.



Imprt the crime_map file that contains the crimes from 1947 till 2020 in NY city

In [598]:
crime_map = pd.read_csv('Crime_Map_.csv')

In [599]:
crime_map.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon
0,466502077,75,BROOKLYN,03/30/2020,17:30:00,03/31/2020,06:53:00,COMPLETED,NaN,N.Y. POLICE DEPT,341,MISDEMEANOR,OPPOSITE OF,PETIT LARCENY,NaN,321.0,"LARCENY,PETIT FROM AUTO",STREET,03/31/2020,1018496,178657,40.656991,-73.876574,"(40.65699087900003, -73.87657444799999)"
1,303191835,77,BROOKLYN,03/28/2020,19:30:00,03/28/2020,20:30:00,COMPLETED,NaN,N.Y. POLICE DEPT,104,FELONY,OPPOSITE OF,RAPE,NaN,157.0,RAPE 1,STREET,03/30/2020,1003606,185050,40.674583,-73.930222,"(40.67458330800008, -73.93022154099998)"
2,735488557,43,BRONX,03/29/2020,14:10:00,NaN,NaN,COMPLETED,NaN,N.Y. POLICE DEPT,341,MISDEMEANOR,INSIDE,PETIT LARCENY,NaN,333.0,"LARCENY,PETIT FROM STORE-SHOPL",DRUG STORE,03/29/2020,1019853,241853,40.830443,-73.871349,"(40.83044253800006, -73.871349147)"
3,315962428,40,BRONX,03/29/2020,07:10:00,03/29/2020,07:16:00,COMPLETED,NaN,N.Y. POLICE DEPT,341,MISDEMEANOR,INSIDE,PETIT LARCENY,NaN,333.0,"LARCENY,PETIT FROM STORE-SHOPL",GROCERY/BODEGA,03/29/2020,1007236,237260,40.817878,-73.916957,"(40.817877907000025, -73.91695668199996)"
4,165437868,114,QUEENS,03/27/2020,13:15:00,03/27/2020,14:00:00,COMPLETED,NaN,N.Y. POLICE DEPT,344,MISDEMEANOR,INSIDE,ASSAULT 3 & RELATED OFFENSES,NaN,101.0,ASSAULT 3,OTHER,03/27/2020,1002018,213258,40.752011,-73.935872,"(40.75201086000004, -73.93587196099996)"


Clean the data and reduce it to the important columns 

In [600]:
crime_map=crime_map[['BORO_NM' ,'LAW_CAT_CD', 'OFNS_DESC','PD_DESC', 'Latitude', 'Longitude' ]]

In [601]:
crime_map.head()

,BORO_NM,LAW_CAT_CD,OFNS_DESC,PD_DESC,Latitude,Longitude
0,BROOKLYN,MISDEMEANOR,PETIT LARCENY,"LARCENY,PETIT FROM AUTO",40.656991,-73.876574
1,BROOKLYN,FELONY,RAPE,RAPE 1,40.674583,-73.930222
2,BRONX,MISDEMEANOR,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",40.830443,-73.871349
3,BRONX,MISDEMEANOR,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",40.817878,-73.916957
4,QUEENS,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,40.752011,-73.935872


In [602]:
# replace columns names with more descriptive names 
crime_map.columns=['Boroughs', 'Level_of_offense', 'Offense_desc', 'PD_desc', 'Latitude', 'Longitude']
crime_map.head()

,Boroughs,Level_of_offense,Offense_desc,PD_desc,Latitude,Longitude
0,BROOKLYN,MISDEMEANOR,PETIT LARCENY,"LARCENY,PETIT FROM AUTO",40.656991,-73.876574
1,BROOKLYN,FELONY,RAPE,RAPE 1,40.674583,-73.930222
2,BRONX,MISDEMEANOR,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",40.830443,-73.871349
3,BRONX,MISDEMEANOR,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",40.817878,-73.916957
4,QUEENS,MISDEMEANOR,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,40.752011,-73.935872


In [603]:
# drop na

crime_map = crime_map.dropna()

###  Let's Analyze the Data

In [604]:
# which borough has the highst crime rate
total_crimes = crime_map['Boroughs'].value_counts()


In [605]:
total_crimes=pd.DataFrame(total_crimes)
total_crimes.rename(columns={'Boroughs':'Total_Number_of_Crimes'})

,Total_Number_of_Crimes
BROOKLYN,31028
MANHATTAN,26911
BRONX,23409
QUEENS,22097
STATEN ISLAND,4535


In [606]:
# Distrubitiobn of crimes per level in different Boroughs
crime_map.groupby('Boroughs')['Level_of_offense'].value_counts(normalize=True)

Boroughs       Level_of_offense
BRONX          MISDEMEANOR         0.536973
               FELONY              0.286172
               VIOLATION           0.176855
BROOKLYN       MISDEMEANOR         0.505705
               FELONY              0.333215
               VIOLATION           0.161080
MANHATTAN      MISDEMEANOR         0.549292
               FELONY              0.315559
               VIOLATION           0.135149
QUEENS         MISDEMEANOR         0.505498
               FELONY              0.327737
               VIOLATION           0.166765
STATEN ISLAND  MISDEMEANOR         0.560970
               FELONY              0.240353
               VIOLATION           0.198677
Name: Level_of_offense, dtype: float64

Let's create a dataframe df presenting the number of crime per level in all Boroughs 

In [613]:

df = crime_map.groupby(['Boroughs', 'Level_of_offense'])['Offense_desc'].count()
df=pd.DataFrame(df).reset_index()
df

,Boroughs,Level_of_offense,Offense_desc
0,BRONX,FELONY,6699
1,BRONX,MISDEMEANOR,12570
2,BRONX,VIOLATION,4140
3,BROOKLYN,FELONY,10339
4,BROOKLYN,MISDEMEANOR,15691
5,BROOKLYN,VIOLATION,4998
6,MANHATTAN,FELONY,8492
7,MANHATTAN,MISDEMEANOR,14782
8,MANHATTAN,VIOLATION,3637
9,QUEENS,FELONY,7242


I will also rank the Boroughs based on the level of crimes in the area. I will designate a coefficient of each crime based on the level. 3 for a Felony, 2 for a misdemeanor and 1 for a violation. I will create a new colum 'Total' to compute the total number of points

In [614]:
df.iloc[0,2]

6699

In [615]:
Total=[]
d=[0,0,0,3,3,3,6,6,6,9,9,9,12,12,12]
for i in d:
    Total.append(df.iloc[i,2]*3+df.iloc[i+1,2]*2+df.iloc[i+2,2])

Total=pd.DataFrame(Total)
Total.columns=['Total']

In [616]:
Total

,Total
0,49377
1,49377
2,49377
3,67397
4,67397
5,67397
6,58677
7,58677
8,58677
9,47751


In [617]:
pd.concat([df,Total], axis=1).sort_values('Total', ascending = False)

,Boroughs,Level_of_offense,Offense_desc,Total
3,BROOKLYN,FELONY,10339,67397
4,BROOKLYN,MISDEMEANOR,15691,67397
5,BROOKLYN,VIOLATION,4998,67397
6,MANHATTAN,FELONY,8492,58677
7,MANHATTAN,MISDEMEANOR,14782,58677
8,MANHATTAN,VIOLATION,3637,58677
0,BRONX,FELONY,6699,49377
1,BRONX,MISDEMEANOR,12570,49377
2,BRONX,VIOLATION,4140,49377
9,QUEENS,FELONY,7242,47751


Based on the total number of crime as well as the level of crimes (total points), the ranking is as follows:
    1. BROOKLYN 
    2. MANHATTAN        
    3. BRONX            
    4. QUEENS           
    5. STATEN ISLAND     

## let's explore the venues in NY by Boroughs

Define Foursquare credentials and version 

In [324]:
CLIENT_ID = 'HBECQJHWEKGTQR35BB35TLV1BCTQZSWFYHARNKUZRYFJA2YK' 
CLIENT_SECRET = '5XHVOHLOR1O0MSPXUGYJT432TNLKNBY4H2VI4XAGOTPHKZPX' 
VERSION = '20180605' 


Creat a DataFrame with NY boroughs, lat and lon information

In [325]:
column_names = ['Borough', 'Latitude', 'Longitude'] 

NY = pd.DataFrame(columns=column_names)
NY

,Borough,Latitude,Longitude


Venues in Brooklyn 

In [326]:
List=['BROOKLYN','MANHATTAN', 'BRONX', 'QUEENS','STATEN ISLAND'  ]
geolocator = Nominatim(user_agent="ny_explorer")
for address in List:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    NY=NY.append({ 'Borough':address   ,'Latitude' : latitude, 'Longitude': longitude}, ignore_index=True )


In [327]:
NY

,Borough,Latitude,Longitude
0,BROOKLYN,40.650104,-73.949582
1,MANHATTAN,40.789624,-73.959894
2,BRONX,40.846651,-73.878594
3,QUEENS,40.749824,-73.797634
4,STATEN ISLAND,40.583456,-74.149605


Venues in all NY (searched by boroughs)

In [338]:
LIMIT = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Norough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [339]:
NY_venues = getNearbyVenues(names=NY['Borough'],
                                   latitudes=NY['Latitude'],
                                   longitudes=NY['Longitude']
                                  )

BROOKLYN
MANHATTAN
BRONX
QUEENS
STATEN ISLAND


In [340]:
NY_venues

,Borough,Borough Latitude,Norough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,BROOKLYN,40.650104,-73.949582,Exquisite Delight,40.649091,-73.949243,Caribbean Restaurant
1,BROOKLYN,40.650104,-73.949582,Nostrand Health Foods,40.649092,-73.949243,Juice Bar
2,BROOKLYN,40.650104,-73.949582,Jus Juice,40.652208,-73.949723,Juice Bar
3,BROOKLYN,40.650104,-73.949582,"Courts (Furniture, Electronics, & Appliances)",40.650436,-73.950608,Furniture / Home Store
4,BROOKLYN,40.650104,-73.949582,Bad Gym,40.650007,-73.950000,Gym / Fitness Center
5,BROOKLYN,40.650104,-73.949582,Tastee Pattee,40.650850,-73.947838,Caribbean Restaurant
6,BROOKLYN,40.650104,-73.949582,Caribbean Vibes,40.651227,-73.949675,Caribbean Restaurant
7,BROOKLYN,40.650104,-73.949582,Nostrand Donut Shop,40.651030,-73.949457,Donut Shop
8,BROOKLYN,40.650104,-73.949582,The Yoga Studio,40.650000,-73.950000,Yoga Studio
9,BROOKLYN,40.650104,-73.949582,Nio's Trinidad Roti Shop,40.650638,-73.952338,Caribbean Restaurant


In [336]:
NY_venues.shape

(88, 7)

In [323]:
NY_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
BRONX,25,25,25,25,25,25
BROOKLYN,23,23,23,23,23,23
MANHATTAN,30,30,30,30,30,30
QUEENS,5,5,5,5,5,5
STATEN ISLAND,5,5,5,5,5,5


# The number of venues is not enough. I will need to search by neighborhoods

## Venues in BROOKLYN

In [363]:
# dataframe of all NY neighborhoods
NY_neighborhoods = pd.read_csv('NY_neighborhoods.csv', index_col=0)
NY_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [364]:
# dataframe of Brooklyn neighborhoods
Brooklyn_neighborhoods=NY_neighborhoods[NY_neighborhoods['Borough']=='Brooklyn'].reset_index(drop=True)
Brooklyn_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


### Explore venues in Brooklyn

In [365]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [366]:
brooklyn_venues = getNearbyVenues(names=Brooklyn_neighborhoods['Neighborhood'],
                                   latitudes=Brooklyn_neighborhoods['Latitude'],
                                   longitudes=Brooklyn_neighborhoods['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [388]:
brooklyn_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,The Bookmark Shoppe,40.624577,-74.030562,Bookstore


In [533]:
brooklyn_venues_sorted = brooklyn_venues.groupby('Venue Category').count()['Neighborhood'].sort_values(ascending=False)

In [534]:
brooklyn_venues_sorted=pd.DataFrame(brooklyn_venues_sorted).rename(columns={'Neighborhood':'Brooklyn'})

In [535]:
brooklyn_venues_sorted

,Brooklyn
Venue Category,
Pizza Place,130
Coffee Shop,98
Bar,84
Deli / Bodega,68
Italian Restaurant,64
Bakery,63
Grocery Store,57
Chinese Restaurant,53
Mexican Restaurant,49


## Venues in MANHATTAN

In [385]:
# dataframe of Manhatan neighborhoods
Manhattan_neighborhoods=NY_neighborhoods[NY_neighborhoods['Borough']=='Manhattan'].reset_index(drop=True)
Manhattan_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


### Explore Venues in Manhattan

In [387]:
manhattan_venues = getNearbyVenues(names=Manhattan_neighborhoods['Neighborhood'],
                                   latitudes=Manhattan_neighborhoods['Latitude'],
                                   longitudes=Manhattan_neighborhoods['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [389]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [536]:
manhattan_venues_sorted = manhattan_venues.groupby('Venue Category').count()['Neighborhood'].sort_values(ascending=False)

In [537]:
manhattan_venues_sorted=pd.DataFrame(manhattan_venues_sorted).rename(columns={'Neighborhood':'Manhattan'})

In [538]:
manhattan_venues_sorted

,Manhattan
Venue Category,
Coffee Shop,143
Italian Restaurant,112
Café,86
Pizza Place,85
American Restaurant,71
Park,71
Bakery,70
Hotel,67
Gym / Fitness Center,60


## Venues in BRONX

In [392]:
# dataframe of Bronx neighborhoods
Bronx_neighborhoods=NY_neighborhoods[NY_neighborhoods['Borough']=='Bronx'].reset_index(drop=True)
Bronx_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Explore venues in Bronx

In [393]:
bronx_venues = getNearbyVenues(names=Bronx_neighborhoods['Neighborhood'],
                                   latitudes=Bronx_neighborhoods['Latitude'],
                                   longitudes=Bronx_neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [394]:
bronx_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [539]:
bronx_venues_sorted = bronx_venues.groupby('Venue Category').count()['Neighborhood'].sort_values(ascending=False)

In [540]:
bronx_venues_sorted=pd.DataFrame(bronx_venues_sorted).rename(columns={'Neighborhood':'Bronx'})

In [541]:
bronx_venues_sorted

,Bronx
Venue Category,
Pizza Place,98
Pharmacy,47
Deli / Bodega,47
Donut Shop,45
Italian Restaurant,39
Chinese Restaurant,37
Sandwich Place,37
Bus Station,36
Grocery Store,35


## Venues in QUEENS

In [397]:
# dataframe of Queens neighborhoods
Queens_neighborhoods=NY_neighborhoods[NY_neighborhoods['Borough']=='Queens'].reset_index(drop=True)
Queens_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


### Explore Venues in QUEENS

In [400]:
queens_venues = getNearbyVenues(names=Queens_neighborhoods['Neighborhood'],
                                   latitudes=Queens_neighborhoods['Latitude'],
                                   longitudes=Queens_neighborhoods['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [457]:
queens_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant


In [542]:
queens_venues_sorted = queens_venues.groupby('Venue Category').count()['Neighborhood'].sort_values(ascending=False)

In [543]:
queens_venues_sorted=pd.DataFrame(queens_venues_sorted).rename(columns={'Neighborhood':'Queens'})

In [544]:
queens_venues_sorted

,Queens
Venue Category,
Pizza Place,89
Deli / Bodega,71
Chinese Restaurant,57
Bakery,56
Donut Shop,52
Pharmacy,49
Bar,48
Bank,46
Sandwich Place,43


## Venues in STATEN ISLAND 

In [515]:
# dataframe of Staten_Island neighborhoods
Staten_Island_neighborhoods=NY_neighborhoods[NY_neighborhoods['Borough']=='Staten Island'].reset_index(drop=True)
Staten_Island_neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


### Explore venues in Staten Island

In [407]:
staten_island_venues = getNearbyVenues(names=Staten_Island_neighborhoods['Neighborhood'],
                                   latitudes=Staten_Island_neighborhoods['Latitude'],
                                   longitudes=Staten_Island_neighborhoods['Longitude']
                                  )

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills


In [451]:
staten_island_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,St. George,40.644982,-74.079353,A&S Pizzeria,40.643940,-74.077626,Pizza Place
1,St. George,40.644982,-74.079353,Beso,40.643306,-74.076508,Tapas Restaurant
2,St. George,40.644982,-74.079353,Staten Island September 11 Memorial,40.646767,-74.076510,Monument / Landmark
3,St. George,40.644982,-74.079353,Richmond County Bank Ballpark,40.645056,-74.076864,Baseball Stadium
4,St. George,40.644982,-74.079353,Shake Shack,40.643660,-74.075891,Burger Joint


In [545]:
staten_island_venues_sorted = staten_island_venues.groupby('Venue Category').count()['Neighborhood'].sort_values(ascending=False)

In [546]:
staten_island_venues_sorted=pd.DataFrame(staten_island_venues_sorted).rename(columns={'Neighborhood':'Staten Island'})

In [547]:
staten_island_venues_sorted

,Staten Island
Venue Category,
Pizza Place,57
Italian Restaurant,42
Bus Stop,42
Deli / Bodega,40
Bagel Shop,22
Pharmacy,21
Sandwich Place,20
Grocery Store,19
Coffee Shop,19


In [549]:
Boroughs_venues = pd.concat([brooklyn_venues_sorted,manhattan_venues_sorted,bronx_venues_sorted,queens_venues_sorted, staten_island_venues_sorted], axis=1)

In [619]:
Boroughs_venues.dropna().head(20)

,Brooklyn,Manhattan,Bronx,Queens,Staten Island
Pizza Place,130.0,85.0,98.0,89.0,57.0
Coffee Shop,98.0,143.0,14.0,36.0,19.0
Bar,84.0,59.0,16.0,48.0,16.0
Deli / Bodega,68.0,33.0,47.0,71.0,40.0
Italian Restaurant,64.0,112.0,39.0,38.0,42.0
Bakery,63.0,70.0,23.0,56.0,8.0
Grocery Store,57.0,33.0,35.0,38.0,19.0
Chinese Restaurant,53.0,41.0,37.0,57.0,18.0
Mexican Restaurant,49.0,57.0,23.0,32.0,13.0
Ice Cream Shop,48.0,39.0,14.0,24.0,14.0
